In [1]:
import nltk
import datetime as datetime 
import pickle 
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
mw_dataset_no_dup_no_troll = pd.read_csv('twitter_noninfluencers_withemotioncat.csv', encoding='latin1')

In [3]:
len(mw_dataset_no_dup_no_troll)

199979

In [4]:
mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,ID,keywords,Date,tweet_timestamp,Tweet,anger,fear,sadness,joy,valence,user_id,user_status_count,user_follower,user_friend,influencer_score,info_status,info_status_binary,emotion,sentiment,intensity_level
0,7,7,8.0,wuhan,01/28/20,1.580179e+09,First person with the Wuhan virus being treate...,0.483,0.530,0.450,0.251,0.408,1.150286e+18,337.0,1411.0,2011.0,0.701641,info-seeking,1,fear,negative,moderate
1,8,8,9.0,wuhan,01/28/20,1.580179e+09,We have this chat group for everyone in SG off...,0.428,0.523,0.470,0.280,0.428,2.055352e+08,35226.0,336.0,473.0,0.710359,info-seeking,1,fear,negative,moderate
2,9,9,10.0,wuhan,01/28/20,1.580180e+09,chrisvtaylor pakhead RichardBarrow Safe harbor...,0.505,0.594,0.474,0.231,0.390,1.170536e+18,489.0,13.0,221.0,0.058824,info-seeking,1,fear,negative,moderate
3,10,10,11.0,wuhan,01/28/20,1.580180e+09,After the 5th Wuhan imported case confirmed Si...,0.414,0.499,0.471,0.332,0.501,2.913063e+07,44406.0,693.0,1285.0,0.539300,info-seeking,1,fear,neutral,moderate
4,13,13,14.0,wuhan,01/28/20,1.580181e+09,Coronavirus news France to airlift citizens ou...,0.534,0.680,0.549,0.182,0.323,2.271343e+08,117.0,1.0,147.0,0.006803,info-seeking,1,fear,negative,high


In [5]:
mw_dataset_no_dup_no_troll.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ID', 'keywords', 'Date',
       'tweet_timestamp', 'Tweet', 'anger', 'fear', 'sadness', 'joy',
       'valence', 'user_id', 'user_status_count', 'user_follower',
       'user_friend', 'influencer_score', 'info_status', 'info_status_binary',
       'emotion', 'sentiment', 'intensity_level'],
      dtype='object')

In [6]:
mw_dataset_no_dup_no_troll['Tweet'] = mw_dataset_no_dup_no_troll['Tweet'].astype(str)

In [7]:
import inflect
import re
import string
import unicodedata

def clean_text_1(headline):
    headline = headline.lower()
    headline = re.sub('\[.,*?\]','',headline)
    headline = re.sub('[0-9]','',headline)
    headline = re.sub('[%s]' % re.escape(string.punctuation), '', headline)
    headline = re.sub('[‘’“”…–]', '', headline)
    headline = re.sub('\w*\d\n\w*', '', headline)
    return headline

#def replace_contractions(text):
    #"""Replace contractions in string of text"""
    #return contractions.fix(text)

def remove_mentions(sample):
    """Remove mentions from a sample string"""
    return re.sub(r"@\S+", "", str(sample))
    
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", str(sample))

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', str(word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words


def normalize(words):
    #words = replace_numbers(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample= remove_mentions(sample)
    #sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)


round_1 = lambda x: preprocess(x)

In [8]:
mw_dataset_no_dup_no_troll['processed_content'] = mw_dataset_no_dup_no_troll['Tweet'].apply(round_1)

In [9]:
mw_dataset_no_dup_no_troll['processed_content']=mw_dataset_no_dup_no_troll['processed_content'].apply(' '.join)

In [10]:
mw_dataset_no_dup_no_troll['processed_content'].head()

0    first person with the wuhan virus being treate...
1    we have this chat group for everyone in sg off...
2    chrisvtaylor pakhead richardbarrow safe harbor...
3    after the 5th wuhan imported case confirmed si...
4    coronavirus news france to airlift citizens ou...
Name: processed_content, dtype: object

In [11]:

mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,ID,keywords,Date,tweet_timestamp,Tweet,anger,fear,sadness,joy,valence,user_id,user_status_count,user_follower,user_friend,influencer_score,info_status,info_status_binary,emotion,sentiment,intensity_level,processed_content
0,7,7,8.0,wuhan,01/28/20,1.580179e+09,First person with the Wuhan virus being treate...,0.483,0.530,0.450,0.251,0.408,1.150286e+18,337.0,1411.0,2011.0,0.701641,info-seeking,1,fear,negative,moderate,first person with the wuhan virus being treate...
1,8,8,9.0,wuhan,01/28/20,1.580179e+09,We have this chat group for everyone in SG off...,0.428,0.523,0.470,0.280,0.428,2.055352e+08,35226.0,336.0,473.0,0.710359,info-seeking,1,fear,negative,moderate,we have this chat group for everyone in sg off...
2,9,9,10.0,wuhan,01/28/20,1.580180e+09,chrisvtaylor pakhead RichardBarrow Safe harbor...,0.505,0.594,0.474,0.231,0.390,1.170536e+18,489.0,13.0,221.0,0.058824,info-seeking,1,fear,negative,moderate,chrisvtaylor pakhead richardbarrow safe harbor...
3,10,10,11.0,wuhan,01/28/20,1.580180e+09,After the 5th Wuhan imported case confirmed Si...,0.414,0.499,0.471,0.332,0.501,2.913063e+07,44406.0,693.0,1285.0,0.539300,info-seeking,1,fear,neutral,moderate,after the 5th wuhan imported case confirmed si...
4,13,13,14.0,wuhan,01/28/20,1.580181e+09,Coronavirus news France to airlift citizens ou...,0.534,0.680,0.549,0.182,0.323,2.271343e+08,117.0,1.0,147.0,0.006803,info-seeking,1,fear,negative,high,coronavirus news france to airlift citizens ou...


In [12]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

#wordnet_lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

#fb_clean['processed_content'] = fb_clean['processed_content'].astype('str').apply(lambda x: filter(None,x.split(" ")))
#fb_clean['processed_content'] = fb_clean['processed_content'].apply(lambda x : [wordnet_lemmatizer.lemmatize(y) for y in x])
#fb_clean['processed_content']=fb_clean['processed_content'].apply(lambda x : " ".join(x))

mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['processed_content'].astype('str').apply(lambda x: filter(None,x.split(" ")))
#df['processed_content'] = df['processed_content'].apply(lambda x : [wordnet_lemmatizer.lemmatize(y) for y in x])
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : [stemmer.stem(y) for y in x])
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : " ".join(x))

In [13]:
mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,ID,keywords,Date,tweet_timestamp,Tweet,anger,fear,sadness,joy,valence,user_id,user_status_count,user_follower,user_friend,influencer_score,info_status,info_status_binary,emotion,sentiment,intensity_level,processed_content,lemmatized_content
0,7,7,8.0,wuhan,01/28/20,1.580179e+09,First person with the Wuhan virus being treate...,0.483,0.530,0.450,0.251,0.408,1.150286e+18,337.0,1411.0,2011.0,0.701641,info-seeking,1,fear,negative,moderate,first person with the wuhan virus being treate...,first person with the wuhan virus be treat by ...
1,8,8,9.0,wuhan,01/28/20,1.580179e+09,We have this chat group for everyone in SG off...,0.428,0.523,0.470,0.280,0.428,2.055352e+08,35226.0,336.0,473.0,0.710359,info-seeking,1,fear,negative,moderate,we have this chat group for everyone in sg off...,we have this chat group for everyon in sg offi...
2,9,9,10.0,wuhan,01/28/20,1.580180e+09,chrisvtaylor pakhead RichardBarrow Safe harbor...,0.505,0.594,0.474,0.231,0.390,1.170536e+18,489.0,13.0,221.0,0.058824,info-seeking,1,fear,negative,moderate,chrisvtaylor pakhead richardbarrow safe harbor...,chrisvtaylor pakhead richardbarrow safe harbor...
3,10,10,11.0,wuhan,01/28/20,1.580180e+09,After the 5th Wuhan imported case confirmed Si...,0.414,0.499,0.471,0.332,0.501,2.913063e+07,44406.0,693.0,1285.0,0.539300,info-seeking,1,fear,neutral,moderate,after the 5th wuhan imported case confirmed si...,after the 5th wuhan import case confirm singap...
4,13,13,14.0,wuhan,01/28/20,1.580181e+09,Coronavirus news France to airlift citizens ou...,0.534,0.680,0.549,0.182,0.323,2.271343e+08,117.0,1.0,147.0,0.006803,info-seeking,1,fear,negative,high,coronavirus news france to airlift citizens ou...,coronavirus news franc to airlift citizen out ...


In [14]:
mw_dataset_no_dup_no_troll.to_csv('twitter_full_noninfluencers_preprocessed_final.csv')

In [ ]:
len(mw_dataset_no_dup_no_troll)